# Домашнее задание к лекции «Продвинутый pandas»

### Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [137]:
import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [138]:
'''
The Ultimate Guide for Column Creation with Pandas DataFrames
https://towardsdatascience.com/the-ultimate-guide-for-column-creation-with-pandas-dataframes-83b8c565110e

Optimum approach for iterating over a DataFrame
https://medium.com/@rtjeannier/pandas-101-cont-9d061cb73bfc
'''
def filtering_traffic_sources(row):
    
    organic = ['yandex', 'google']
    ad = ['paid', 'email']
    
    if row['traffic_source'] in organic:
        return 'organic'
    elif row['traffic_source'] in ad and row['region'] == 'Russia':
        return 'ad'
    elif row['traffic_source'] in ad:
        return 'other'

log['source_type'] = log.apply(filtering_traffic_sources, axis=1)
log.tail()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad
18937,1550094323,b3b634f824,https://host.ru/39fa884393666d45fc,Russia,7d27a58fb8,email,ad


### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [139]:
def head_line_by_line(f):
    for i, line in enumerate(f):
        print(i, line.strip())

        if i > 4:
            break
            
with open('URLs.txt') as f:
    head_line_by_line(f)

0 url
1 /world/
2 /latest/
3 /?updated=top
4 /politics/36188461-s-marta-zhizn-rossiyan-suschestvenno-izmenitsya-iz-za-novyh-zakonov/
5 /world/36007585-tramp-pridumal-kak-reshit-ukrainskiy-vopros/


In [140]:
urls = pd.read_csv('URLs.txt', sep=' ')
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [141]:
'''
Количество повторений символов
https://ravesli.com/urok-7-kolichestvo-povtorenij-simvola-regexp/
'''
urls[urls['url'].str.contains('/[0-9]{8}-')].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


### Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [142]:
with open('./ml-latest-small/ratings.csv') as f:
    head_line_by_line(f)

0 userId,movieId,rating,timestamp
1 1,31,2.5,1260759144
2 1,1029,3.0,1260759179
3 1,1061,3.0,1260759182
4 1,1129,2.0,1260759185
5 1,1172,4.0,1260759205


In [143]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv', sep=',')
ratings.sample(5)

,userId,movieId,rating,timestamp
56104,405,3653,5.0,1097696309
65146,463,3260,4.0,1049912626
83451,564,1275,1.0,974838752
21873,151,315,2.0,847296945
35808,257,26776,3.5,1348289356


In [144]:
'''
Pandas: Group by time and get the difference of the max and min values in each group
https://arccoder.medium.com/pandas-group-by-time-and-get-the-difference-of-the-max-and-min-values-in-each-group-579c2d49c27b
'''
df = ratings.groupby(pd.Grouper(key='userId')) \
    .apply(lambda x: max(x['timestamp']) - min(x['timestamp'])) \
    .reset_index(name='lifetime')

df.head(5)

,userId,lifetime
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101


### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).

**Дана статистика услуг перевозок клиентов компании по типам:**

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

**Необходимо сформировать две таблицы:**

1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [145]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [146]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [147]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [148]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [149]:
'''
Подготовка данных в pandas / pd 10
https://pythonru.com/biblioteki/podgotovka-dannyh-v-pandas-pd-10

select columns by name
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html
'''
client_ids = client_base.filter(items=['client_id'])
client_ids

,client_id
0,111
1,112
2,113
3,114
4,115
5,116
6,117
7,118


In [150]:
'''
Combining DataFrames with Pandas
https://datacarpentry.org/python-ecology-lesson/05-merging-data/
'''
merged_left = pd.merge(left=client_ids, right=air, how='left', left_on='client_id', right_on='client_id')
merged_left

,client_id,air_revenue
0,111,NaN
1,112,NaN
2,113,NaN
3,114,NaN
4,115,81.0
5,116,4.0
6,117,13.0
7,118,173.0


In [151]:
merged_left = pd.merge(left=merged_left, right=auto, how='left', left_on='client_id', right_on='client_id')
merged_left

,client_id,air_revenue,auto_revenue
0,111,NaN,NaN
1,112,NaN,NaN
2,113,NaN,57483.0
3,114,NaN,83.0
4,115,81.0,912.0
5,116,4.0,4834.0
6,117,13.0,98.0
7,118,173.0,NaN


In [152]:
transportation_service_stats = pd.merge(left=merged_left, right=rzd, how='left', left_on='client_id', right_on='client_id')

transportation_service_stats

,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,NaN,NaN,1093.0
1,112,NaN,NaN,2810.0
2,113,NaN,57483.0,10283.0
3,114,NaN,83.0,5774.0
4,115,81.0,912.0,981.0
5,116,4.0,4834.0,NaN
6,117,13.0,98.0,NaN
7,118,173.0,NaN,NaN


In [153]:
pd.merge(left=client_base, right=transportation_service_stats, how='left', left_on='client_id', right_on='client_id')

,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,NaN,57483.0,10283.0
3,114,Мира 14,NaN,83.0,5774.0
4,115,ЗЖБИиДК 1,81.0,912.0,981.0
5,116,Строителей 18,4.0,4834.0,NaN
6,117,Панфиловская 33,13.0,98.0,NaN
7,118,Мастеркова 4,173.0,NaN,NaN
